In [1]:
import numpy as np

## Parent Agent Class

The class defined the mechanics of any agents in the simulation

In [2]:
class TradingAgent:
    def __init__(self, stock, cash):
        self.X = stock.copy()
        self.cash = cash.copy()
        self.min_leverage = 1
        self.max_leverage = 8
        self.action = np.zeros(np.shape(X))
        self.transaction = np.zeros(np.shape(X))
        
    def make_transaction(self, S, current_step):
        return np.random.randint(-2, 3)
    
    def trade(self, S, current_step):
        self.X[current_step] = self.X[current_step - 1] + self.make_transaction(S, current_step)
        self.cash[current_step] = self.cash[current_step - 1] - self.make_transaction(S, current_step) * S[current_step - 1]
            
    def get_stock(self):
        return self.X
    
    def get_cash(self):
        return self.cash
    
    def get_value(self, S):
        return [x * s + c for x, s, c in zip(self.X, S, self.cash)]
    
    def get_action(self):
        return self.action
    
    def get_transaction(self):
        return self.transaction

## Random Agents

This type of agents will place random orders in each time steps. The purpose of creating Random Agents are:
* To fill the market with a fixed number of agents
* Generate noise

In [3]:
class RandomTrader(TradingAgent):  
    def make_transaction(self, S, current_step):
        buyOrSell = np.random.randint(0, 2)
        if buyOrSell == 0:
            self.transaction[current_step] = self.buy(S, current_step)
            return self.buy(S, current_step)
        elif buyOrSell == 1:
            self.transaction[current_step] = self.sell(S, current_step)
            return self.sell(S, current_step)
    
    def buy(self, S, current_step):
        self.action[current_step] = 1
        if self.cash[current_step - 1] >= S[current_step - 1] * 2:
            return np.random.randint(-2, 1)
        else:
            return 0
    
    def sell(self, S, current_step):
        self.action[current_step] = -1
        if self.X[current_step - 1] >= 2:
            return np.random.randint(0, 3)
        else:
            return 0

## Trend Following Traders

This agents will make decisions following the trend in the market price. It will buy when the price goes above moving average price in a specific time window and sell otherwise

In [4]:
class TrendFollower(TradingAgent):
    def make_transaction(self, S, current_step):
        if current_step <= 3:
            return 0
        mean = sum(S[current_step - 4:current_step - 1])/3
        if S[current_step - 1] < mean:
            self.transaction[current_step] = self.sell(S, current_step)
            return self.sell(S, current_step)
        elif S[current_step - 1] > mean:
            self.transaction[current_step] = self.buy(S, current_step)
            return self.buy(S, current_step)
        else:
            return 0
        
    def buy(self, S, current_step):
        self.action[current_step] = 1
        totalValue = X[current_step - 1] * S[current_step - 1] + cash[current_step - 1]
        return np.floor((self.max_leverage / (self.max_leverage + 1)) * totalValue / S[current_step - 1]) - self.X[current_step - 1]
    
    def sell(self, S, current_step):
        self.action[current_step] = -1
        totalValue = X[current_step - 1] * S[current_step - 1] + cash[current_step - 1]
        return np.ceil((self.min_leverage / (self.min_leverage + 1)) * totalValue / S[current_step - 1]) - self.X[current_step - 1]

## Mean Reversion

This agent algorithm is the opposition of trend following. It will goes against the market price trend. This algorithm will often lead to a more active type of agent, as its placing of orders will drive the price in the opposite direction, making it more vulnerable to trading activities.

In [5]:
class MeanReversion(TradingAgent):
    def make_transaction(self, S, current_step):
        if current_step <= 3:
            return 0
        mean = sum(S[current_step - 4:current_step - 1])/3
        if S[current_step - 1] > mean:
            self.transaction[current_step] = self.sell(S, current_step)
            return self.sell(S, current_step)
        elif S[current_step - 1] < mean:
            self.transaction[current_step] = self.buy(S, current_step)
            return self.buy(S, current_step)
        else:
            return 0
        
    def buy(self, S, current_step):
        self.action[current_step] = 1
        totalValue = X[current_step - 1] * S[current_step - 1] + cash[current_step - 1]
        return np.floor((self.max_leverage / (self.max_leverage + 1)) * totalValue / S[current_step - 1]) - self.X[current_step - 1]
    
    def sell(self, S, current_step):
        self.action[current_step] = -1
        totalValue = X[current_step - 1] * S[current_step - 1] + cash[current_step - 1]
        return np.ceil((self.min_leverage / (self.min_leverage + 1)) * totalValue / S[current_step - 1]) - self.X[current_step - 1]